In [1]:
import sys
sys.path.append('../src')
# from dataloader import create_dataloaders
from dataloader_ER import create_dataloaders,create_dataloaders_WebTable
# from lora_model import LORAEngine,LORAEngineDeberta,LORAEngineDebertaMultiClass
from lora_model_CTA import LORAEngineDebertaMultiClass

# from influence import IFEngine
from influence_hyperinf import IFEngine

import numpy as np
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [2]:
# model_name_or_path="roberta-large"
# model_name_or_path="/home/yanmy/roberta-base"
model_name_or_path="../../model/deberta-v3-base"
# model_name_or_path="../../model/roberta-large"
task="WebTable"
# task = "qnli"
noise_ratio=0
batch_size=64
# target_modules=["query_proj", "key_proj", "value_proj"] ## deberta, for roberta, use ["value"]
# target_modules = ['value']
target_modules=["value_proj"] ## deberta, for roberta, use ["value"]

device="cuda:7"
num_epochs=10
lr=3e-4

In [3]:
dataloader_outputs = create_dataloaders_WebTable(model_name_or_path=model_name_or_path,
                                           task=task,
                                           batch_size=batch_size,
                                           train_file = '../datasets/webtable_IF/train.csv',
                                           valid_file = '../datasets/webtable_IF/valid.csv',
                                           test_file = '../datasets/webtable_IF/test.csv',
                                           max_length = 256,
                                        #    select_index=np.arange(0,5000,1)
                                           )

train_dataloader, eval_dataloader,test_dataloader, tokenized_datasets, collate_fn, num_labels, int2label = dataloader_outputs


# lora_engine = LORAEngine(model_name_or_path=model_name_or_path,
#                             target_modules=target_modules,
#                             train_dataloader=train_dataloader,
#                             eval_dataloader=eval_dataloader,
#                             device=device,
#                             num_epochs=num_epochs,
#                             lr=lr,
#                             low_rank=8, 
#                             task=task)

lora_engine = LORAEngineDebertaMultiClass(model_name_or_path=model_name_or_path,
                            target_modules=target_modules,
                            train_dataloader=train_dataloader,
                            # eval_dataloader=eval_dataloader,
                            eval_dataloader=eval_dataloader,
                            test_dataloader = test_dataloader,
                            device=device,
                            num_epochs=num_epochs,
                            lr=lr,
                            num_labels = num_labels,
                            low_rank=16, 
                            task=task,
                            save_path = '../models/mrpc',
                            valid_each_epoch=False,
                            cal_grad_per_sample = True,
                            grad_epoch = [5,7,9])

Map:   0%|          | 0/45000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4106 [00:00<?, ? examples/s]

Map:   0%|          | 0/51648 [00:00<?, ? examples/s]

In [4]:
lora_engine.build_LORA_model()
tr_grad_dict,val_grad_dict = lora_engine.train_LORA_model()
import torch
torch.save(tr_grad_dict,'../output/CTA/webtable/tr_grad_dict_all.pkl')
torch.save(val_grad_dict,'../output/CTA/webtable/val_grad_dict_all.pkl')

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at ../../model/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 458,709 || all params: 185,044,650 || trainable%: 0.2479
Total Steps:4224


100%|██████████| 704/704 [00:52<00:00, 13.47it/s]


Epoch 1: Training Loss = 3.4880777296017516


100%|██████████| 704/704 [00:50<00:00, 14.05it/s]


...

Epoch 5: Training Loss = 0.8376742519268935


100%|██████████| 704/704 [00:44<00:00, 15.78it/s]


Epoch 6: Training Loss = 0.7673470885069533


100%|██████████| 807/807 [00:56<00:00, 14.31it/s]


Prediction Result on Test Data: Micro F1 = {'f1': 0.6574697955390335}, Macro F1 = {'f1': 0.2502817605279058}


In [ ]:
lora_engine.save_lora('../models/ER-semi-text-c-LoRA')

In [7]:
tr_grad_dict.keys()

dict_keys([])

## save gradient

In [ ]:
import torch
torch.save(tr_grad_dict,'../output/ER/semi-text-c/tr_grad_dict_all.pkl')
torch.save(val_grad_dict,'../output/ER/semi-text-c/val_grad_dict_all.pkl')

## 分别计算不同epoch下的分数，然后累加分析

In [ ]:
import torch
from tqdm.notebook import tqdm
tr_grad_dict_all = torch.load('../output/ER/semi-text-c/tr_grad_dict_all.pkl')
val_grad_dict_all = torch.load('../output/ER/semi-text-c/val_grad_dict_all.pkl')

/tmp/ipykernel_3322955/154564480.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tr_grad_dict_all = torch.load('../output/ER/semi-text-c/tr_grad_dict_all.pkl')
/tmp/ipyk

In [5]:
import sys
sys.path.append('../src')
from influence_batch import IFEngine
from tqdm.notebook import tqdm
noise_index = None
IF_device = 'cuda:7' ## 避免爆显存
def compute_val_grad_avg(val_grad_dict):
    # Compute the avg gradient on the validation dataset
    n_val = len(val_grad_dict)
    val_grad_avg_dict={}
    for weight_name in val_grad_dict[0]:
        if weight_name.__contains__('base_model'):
            val_grad_avg_dict[weight_name]=torch.zeros(val_grad_dict[0][weight_name].shape)
            for val_id in val_grad_dict:
                val_grad_avg_dict[weight_name] += val_grad_dict[val_id][weight_name] / n_val
        # else:
        #     val_grad_avg_dict[weight_name] = val_grad_dict[weight_name]
    # val_grad_avg_dict['ids'] = val_grad_dict['ids']
    return val_grad_avg_dict

In [7]:
tr_grad_dict_all = tr_grad_dict
val_grad_dict_all = val_grad_dict

In [9]:
tr_grad_dict_all.keys()

dict_keys([5])

In [8]:
result_df = {}
for epoch in tr_grad_dict_all.keys():
    tr_grad_dict = tr_grad_dict_all[epoch]
    val_grad_dict_avg = compute_val_grad_avg(val_grad_dict_all[epoch]) ## 不把所有的val_grad_dict放到gpu中，节省显存
    for key in tqdm(tr_grad_dict): ## 尝试转为fp16计算
        for kk in tr_grad_dict[key]:
            # if kk!='ids':
                tr_grad_dict[key][kk] = tr_grad_dict[key][kk].to(IF_device)
            # tr_grad_dict[key].pop('ids')

    for key in tqdm(val_grad_dict_avg):
        # for kk in val_grad_dict[key]:
            # if kk!='ids':
                val_grad_dict_avg[key] = val_grad_dict_avg[key].to(IF_device)
    influence_engine = IFEngine(weight_list=val_grad_dict_avg.keys())
    influence_engine.preprocess_gradients(tr_grad_dict, val_grad_dict_avg, noise_index)

    influence_engine.compute_hvps(compute_accurate=False,compute_LiSSA=True)
    influence_engine.compute_IF()
    result_df[epoch] = influence_engine.IF_dict

  0%|          | 0/704 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

Time taken for HyperINF:  1.0962071418762207
Time taken for Hessian-free:  2.1457672119140625e-06


  0%|          | 0/25 [00:00<?, ?it/s]

Time taken for Datainf:  1.935877799987793


  0%|          | 0/25 [00:00<?, ?it/s]

Time taken for LiSSA:  9.297013282775879


In [10]:
torch.save(result_df,'../output/CTA/webtable/IF_dict.pkl')

In [ ]:
result =  pd.DataFrame(result_df[5]['iterative']).T

In [ ]:
result_2 = pd.DataFrame(result_df[2]['iterative']).T
result_4 = pd.DataFrame(result_df[4]['iterative']).T

In [11]:
import pandas as pd


def add_sum_column(df):
    """
    对输入的DataFrame添加一列，列的值为所有列名包含'model'的列的元素之和。

    参数:
    df (pd.DataFrame): 输入的DataFrame对象

    返回:
    pd.DataFrame: 添加了新列后的DataFrame对象
    """
    sum_cols = [col for col in df.columns if 'model' in col]
    df['sum_model_cols'] = df[sum_cols].sum(axis=1)
    return df
# result_2 = add_sum_column(result_2)
# result_4 = add_sum_column(result_4)
result =  pd.DataFrame(result_df[5]['iterative']).T
result = add_sum_column(result)

In [12]:
import numpy as np
sample_IF = np.zeros(45000)
for index,row in result.iterrows():
    ids = row['ids']
    sum = row['sum_model_cols']
    for id in ids:
        sample_IF[id] += sum


In [ ]:
import numpy as np
sample_IF = np.zeros(45000)
for index,row in result_2.iterrows():
    ids = row['ids']
    sum = row['sum_model_cols']
    for id in ids:
        sample_IF[id] += sum
for index,row in result_4.iterrows():
    ids = row['ids']
    sum = row['sum_model_cols']
    for id in ids:
        sample_IF[id] += sum

In [15]:
select_index = np.argsort(sample_IF)[:4500]

## RETRAIN

In [ ]:
import torch
import numpy as np
import pandas as pd
result_df = torch.load('../output/ER/semi-text-c/IF_dict.pkl')
# sample_IF = {}
# # for i in range(4500):
# #     sample_IF[i] = 0
# for epoch in result_df.keys():
#     for method in ['iterative']:
#         for batch_id in result_df[epoch][method].keys(): ## 选出来所有的batch
#             IF_score = result_df[epoch][method][batch_id]
#             ids = IF_score['ids']
result_2 = pd.DataFrame(result_df[2]['iterative']).T
result_4 = pd.DataFrame(result_df[4]['iterative']).T

import pandas as pd


def add_sum_column(df):
    """
    对输入的DataFrame添加一列，列的值为所有列名包含'model'的列的元素之和。

    参数:
    df (pd.DataFrame): 输入的DataFrame对象

    返回:
    pd.DataFrame: 添加了新列后的DataFrame对象
    """
    sum_cols = [col for col in df.columns if 'model' in col]
    df['sum_model_cols'] = df[sum_cols].sum(axis=1)
    return df
result_2 = add_sum_column(result_2)
result_4 = add_sum_column(result_4)


import numpy as np
sample_IF = np.zeros(4500)
for index,row in result_2.iterrows():
    ids = row['ids']
    sum = row['sum_model_cols']
    for id in ids:
        sample_IF[id] += sum
for index,row in result_4.iterrows():
    ids = row['ids']
    sum = row['sum_model_cols']
    for id in ids:
        sample_IF[id] += sum

/tmp/ipykernel_3336774/2073020919.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  result_df = torch.load('../output/ER/semi-text-c/IF_dict.pkl')


In [ ]:
select_index = np.argsort(sample_IF)[:500]

### Ablation


In [ ]:
import sys
sys.path.append('../src')
# from dataloader import create_dataloaders
from dataloader_ER import create_dataloaders
# from lora_model import LORAEngine,LORAEngineDeberta,LORAEngineDebertaMultiClass
from lora_model_vmap import LORAEngineDebertaMultiClass

# from influence import IFEngine
from influence_hyperinf import IFEngine

import numpy as np
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings("ignore")
# model_name_or_path="roberta-large"
# model_name_or_path="/home/yanmy/roberta-base"
model_name_or_path="../../model/deberta-v3-base"
# model_name_or_path="../../model/roberta-large"
task="ER"
# task = "qnli"
noise_ratio=0
batch_size=16
# target_modules=["query_proj", "key_proj", "value_proj"] ## deberta, for roberta, use ["value"]
# target_modules = ['value']
target_modules=["value_proj"] ## deberta, for roberta, use ["value"]

device="cuda:7"
num_epochs=15
lr=3e-4

In [ ]:
dataloader_outputs = create_dataloaders(model_name_or_path=model_name_or_path,
                                           task=task,
                                           noise_ratio=noise_ratio,
                                           batch_size=batch_size,
                                           train_file = '../ER/semi-text-c/train.json',
                                           valid_file = '../ER/semi-text-c/valid.json',
                                           test_file = '../ER/semi-text-c/test.json',
                                           max_length=256,
                                           select_index = select_index)
# train_dataloader, eval_dataloader, noise_index, tokenized_datasets, collate_fn=dataloader_outputs
train_dataloader, eval_dataloader,test_dataloader, tokenized_datasets, collate_fn=dataloader_outputs


# lora_engine = LORAEngine(model_name_or_path=model_name_or_path,
#                             target_modules=target_modules,
#                             train_dataloader=train_dataloader,
#                             eval_dataloader=eval_dataloader,
#                             device=device,
#                             num_epochs=num_epochs,
#                             lr=lr,
#                             low_rank=8, 
#                             task=task)

lora_engine = LORAEngineDebertaMultiClass(model_name_or_path=model_name_or_path,
                            target_modules=target_modules,
                            train_dataloader=train_dataloader,
                            # eval_dataloader=eval_dataloader,
                            eval_dataloader=eval_dataloader,
                            test_dataloader = test_dataloader,
                            device=device,
                            num_epochs=num_epochs,
                            lr=lr,
                            low_rank=16, 
                            task=task,
                            save_path = '../models/mrpc',
                            valid_each_epoch=False,
                            cal_grad_per_sample = False)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/737 [00:00<?, ? examples/s]

Map:   0%|          | 0/4179 [00:00<?, ? examples/s]

In [ ]:
lora_engine.build_LORA_model()
lora_engine.train_LORA_model()

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at ../../model/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 296,450 || all params: 184,720,132 || trainable%: 0.1605
Total Steps:480


100%|██████████| 32/32 [00:02<00:00, 11.41it/s]


Epoch 1: Training Loss = 0.6829902809113264


100%|██████████| 32/32 [00:02<00:00, 13.88it/s]


...

100%|██████████| 32/32 [00:02<00:00, 13.75it/s]


Epoch 15: Training Loss = 0.1355522975209169


100%|██████████| 262/262 [00:18<00:00, 13.82it/s]

Prediction Result on Test Data: {'f1': 0.7880910683012259}


({}, {})

In [ ]:
{'f1': 0.8182701652089407} 1000
{'f1': 0.7937062937062938} 500

## CTA

In [23]:
result =  pd.DataFrame(result_df[5]['proposed']).T
result = add_sum_column(result)

import numpy as np
sample_IF = np.zeros(45000)
for index,row in result.iterrows():
    ids = row['ids']
    sum = row['sum_model_cols']
    for id in ids:
        sample_IF[id] += sum
select_index = np.argsort(sample_IF)[:9000]

In [24]:
dataloader_outputs = create_dataloaders_WebTable(model_name_or_path=model_name_or_path,
                                           task=task,
                                           batch_size=batch_size,
                                           train_file = '../datasets/webtable_IF/train.csv',
                                           valid_file = '../datasets/webtable_IF/valid.csv',
                                           test_file = '../datasets/webtable_IF/test.csv',
                                           max_length = 256,
                                           select_index=select_index
                                           )

train_dataloader, eval_dataloader,test_dataloader, tokenized_datasets, collate_fn, num_labels, int2label = dataloader_outputs


# lora_engine = LORAEngine(model_name_or_path=model_name_or_path,
#                             target_modules=target_modules,
#                             train_dataloader=train_dataloader,
#                             eval_dataloader=eval_dataloader,
#                             device=device,
#                             num_epochs=num_epochs,
#                             lr=lr,
#                             low_rank=8, 
#                             task=task)

lora_engine = LORAEngineDebertaMultiClass(model_name_or_path=model_name_or_path,
                            target_modules=target_modules,
                            train_dataloader=train_dataloader,
                            # eval_dataloader=eval_dataloader,
                            eval_dataloader=eval_dataloader,
                            test_dataloader = test_dataloader,
                            device=device,
                            num_epochs=10,
                            lr=lr,
                            num_labels = num_labels,
                            low_rank=16, 
                            task=task,
                            save_path = '../models/mrpc',
                            valid_each_epoch=False,
                            cal_grad_per_sample = False,
                            grad_epoch = [5,7,9])

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4106 [00:00<?, ? examples/s]

Map:   0%|          | 0/51648 [00:00<?, ? examples/s]

In [25]:
lora_engine.build_LORA_model()
lora_engine.train_LORA_model()

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at ../../model/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 458,709 || all params: 185,044,650 || trainable%: 0.2479
Total Steps:1410


100%|██████████| 141/141 [00:08<00:00, 15.91it/s]


Epoch 1: Training Loss = 4.622079008859946


100%|██████████| 141/141 [00:08<00:00, 16.14it/s]


...

100%|██████████| 141/141 [00:08<00:00, 15.78it/s]


Epoch 10: Training Loss = 1.8906240175801812


100%|██████████| 807/807 [00:56<00:00, 14.31it/s]


Prediction Result on Test Data: Micro F1 = {'f1': 0.4569973667905824}, Macro F1 = {'f1': 0.09202929732558915}


({}, {})

### CTA数据需要重构？
